Running inference on image pairs
============================

If you want to use a pre-trained PWC-Net model on your own set of images, you can pass a list of image pairs to a `ModelPWCNet` object using its  `predict_from_img_pairs()` method, as demonstrated here.

Below, look for `TODO` references and customize this notebook based on your own needs.

In [17]:
"""
pwcnet_predict_from_img_pairs.ipynb

Run inference on a list of images pairs.

Written by Phil Ferriere

Licensed under the MIT License (see LICENSE for details)
"""
from __future__ import absolute_import, division, print_function
from copy import deepcopy
from imageio import imread
from model_pwcnet import ModelPWCNet, _DEFAULT_PWCNET_TEST_OPTIONS
from visualize import display_img_pairs_w_flows
from optflow import flow_to_img
import numpy as np

In [21]:
from glob import glob
import os.path as osp
import os

def get_files(d, pattern, sort=True):
    files = glob(osp.join(d, pattern))
    files = [f for f in files if osp.isfile(f)]
    if sort:
        files.sort(key=lambda x: int(x.split("/")[-1].split(".")[0]))
    return files

def get_subdirs(d, nonempty=False, sort=True):
    subdirs = [f.path for f in os.scandir(d) if f.is_dir()]
    if nonempty:
        subdirs = [f for f in subdirs if not is_folder_empty(f)]
    if sort:
        subdirs.sort(key=lambda x: x.split("/")[-1])
    return subdirs

In [3]:
# TODO: Set device to use for inference
# Here, we're using a GPU (use '/device:CPU:0' to run inference on the CPU)
gpu_devices = ['/device:CPU:0']  
controller = '/device:CPU:0'

# TODO: Set the path to the trained model (make sure you've downloaded it first from http://bit.ly/tfoptflow)
ckpt_path = './models/pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-595000'

In [5]:
# Configure the model for inference, starting with the default options
nn_opts = deepcopy(_DEFAULT_PWCNET_TEST_OPTIONS)
nn_opts['verbose'] = True
nn_opts['ckpt_path'] = ckpt_path
nn_opts['batch_size'] = 1
nn_opts['gpu_devices'] = gpu_devices
nn_opts['controller'] = controller

# We're running the PWC-Net-large model in quarter-resolution mode
# That is, with a 6 level pyramid, and upsampling of level 2 by 4 in each dimension as the final flow prediction
nn_opts['use_dense_cx'] = True
nn_opts['use_res_cx'] = True
nn_opts['pyr_lvls'] = 6
nn_opts['flow_pred_lvl'] = 2

# The size of the images in this dataset are not multiples of 64, while the model generates flows padded to multiples
# of 64. Hence, we need to crop the predicted flows to their original size
nn_opts['adapt_info'] = (1, 436, 1024, 2)

In [6]:
# Instantiate the model in inference mode and display the model configuration
nn = ModelPWCNet(mode='test', options=nn_opts)
nn.print_config()







Building model...


Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use `tf.keras.layers.Conv2DTranspose` instead.

... model built.

Loading model checkpoint ./models/pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-595000 for eval or testing...

INFO:tensorflow:Restoring parameters from ./models/pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-595000
... model loaded

Model Configuration:
  verbose                True
  ckpt_path              ./models/pwcnet-lg-6-2-multisteps-chairsthingsmix/pwcnet.ckpt-595000
  x_dtype                <dtype: 'float32'>
  x_shape                [2, None, None, 3]
  y_dtype                <dtype: 'float32'>
  y_shape                [None, None, 2]
  gpu_devices            ['/device:CPU:0']
  controller             /device:CPU:0
  batch_size             1
  use_tf_data            True
  use_mixed_precision    False
  pyr_lvls

In [31]:
flows = []
frames = []
root_dir = '/Users/kevin/repos/htl/htl/data/Penn_Action/valid/baseball_pitch/'
for i, folder in enumerate(get_subdirs(root_dir)):
    if i > 9:
        break
    files = get_files(folder, "*.jpg")
    img_pairs = []
    for pair in range(0, len(files)-1):
        image_path1 = files[pair]
        image_path2 = files[pair+1]
        image1, image2 = imread(image_path1), imread(image_path2)
        img_pairs.append((image1, image2))
    pred_labels = nn.predict_from_img_pairs(img_pairs, batch_size=1, verbose=True)
    # display_img_pairs_w_flows(img_pairs, pred_labels)
    frames.append([img[0] for img in img_pairs] + [imread(files[-1])])
    flows.append([flow_to_img(f) for f in pred_labels])

Predicting flows: 100%|###########################################| 108/108 [00:23<00:00,  4.67it/s]


In [32]:
# save embeddings
data = {
    'frames': frames,
    'embs': flows,
}
np.save('/Users/kevin/repos/htl/htl/tmp/flows.npy', data)